# IRRS Lab Session 6: Duplicate detection using simhash

In this session you will:

- implement the simhashing algorithm
- apply it to the `arxiv` abstracts to detect near duplicates among documents in this index

## 1. Simhash-based near-duplicate detection algorithm

We depart from an indexed corpus, namely the one that we have used in previous sessions that contains 58102 abstracts from the
scientific [arXiv.org](https://arxiv.org) repository. The following pseudocode uses $k,m$ as the usual parameters for lsh (locality sensitive hashing)
scheme for gap amplification (repetition of $m$ lsh tables, using $k$ projections each).

  1. For each word $t$ in the vocabulary $V$ we generate $b = k\times m$ random $\pm 1$ signs:
      - generate a unique hash (bitstring) $h_t$ of size $b$
      - let $h'_t$ be the hash above but replacing each 0 with -1.
  
  2. We view each document as a tf-idf weighted sum of its terms. Thus, to hash a document, we sum the weighted contributions of all its terms. Specifically, for each document in our corpus represented by list of non-zero entries $t, w_t$:
      - compute $simhash(d)$ as sum $\sum_t h'_t * w_t$. 
      - convert $simhash(d)$ to bit vector using the sign of entries (positive entries become 1, negative ones become 0)
  
  3. Slice simhashes into $m$ chunks of $k$ bits
     - for each $k$-bit chunk:
        - place the document into the bucket given by the chunk in the appropriate table. Note that we may identify each chunk with an integer index between 0 and 2^k - 1.
  
  4. Similar documents will likely fall on the same side of many hyperplanes, and thus share one or more $k$-bit chunks. Thus, the potential duplicates are pairs of documents that fall into the same bucket in **at least one** hash table. Now we need to
     - compute cosine similarity among these potential duplicates, and
     - show pairs in decreasing cosine similarity order.


---

**Exercise 1:**  

As a little exercise, suppose that a document $d$ contains _two_ non-zero entries, which correspond to terms `bit` with tf-idf weight $0.4$ and `coin` with tf-idf weight $1.2$.
The md5 hash code for `bit` is 1111 and the md5 code for `coin` is 1001 (so $b=4$). Compute the binary simhash for this document with $k=4, m = 1$. (_Hint: result should be 1001_)

**Exercise 1 (Solution):**

$d=[0.4,1.2] \quad ; \quad h'_t(bit)=[1,1,1,1] \quad ; \quad h'_t(coin)=[1,-1,-1,1]$

$simhash(d)=\sum_t h'_t*w_t=[0.4,0.4,0.4,0.4]+[1.2,-1.2,-1.2,1.2]=[1.6,-0.8,-0.8,1.6]$

$simhash_{bit}(d)=[1,0,0,1]$ (replace positive entries of $simhash(d)$ with 1 and negative entries with 0)

---


## 2. Hashing codes for terms in vocabulary

To make your life easier, we provide code for generating hash codes for the terms in the vocabulary.
The given function takes as input the size of the hash, as well as the term itself.

We use a fast deterministic hash (MD5) to get repeatable pseudo-random bits. Even though it is actually deterministic, we assume that in practice its output behaviour is sufficiently similar to that of a truly random hash function.

In [ ]:
import hashlib

def _termhash(x : str, b : int) -> str:
    """returns bitstring of size b based on md5 algorithm"""
    assert b <= 128, 'this encoding scheme supports hashes of length at most 128; try smaller b'
    h = hashlib.md5(x.encode('utf8')).digest()
    return ''.join(format(byte, '08b') for byte in h)[:b]

for w in ['bit', 'coin', 'hola que tal']:
    print(f'hash.md5({w}) = {_termhash(w, 4)}')


## 3. Loading tf-idf for the whole corpus

We provide the document-term matrix with normalized tf-idf weights with a sparse representation.
This is so that further code does not need to check for tf-idf weights against elasticsearch which is very slow.

Please download the corpus data from https://www.cs.upc.edu/~caim/05corpus.pkl and run the following cells to access the data.

In [ ]:
import pickle

fname = '05corpus.pkl'

with open(fname, 'rb') as f:
    corpus = pickle.load(f)

### have a look at the data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
import random

# show contents

print(f"Loaded corpus with {len(corpus)} documents.")
all_document_ids = list(corpus.keys())
print(f"A few document ids are: {sorted(random.sample(all_document_ids, 10))}")
print(f"Each document is a dictionary with term -> tf-idf weights, e.g. the first:")
print(corpus[all_document_ids[0]])

# obtain histogram with nr of terms in each document
values = [len(corpus[id]) for id in all_document_ids]
mu = np.mean(values)
sigma = np.std(values)
plt.figure(figsize=(8, 4))
sns.set_style("whitegrid")
sns.histplot(values, bins=50, edgecolor='black', stat='density', label='Distribution of values')
plt.title(f'Histogram of number of types per document with Gaussian Overlay\nMean = ${mu:.2f}$, Std Dev = ${sigma:.2f}$', fontsize=12)
xmin, xmax = plt.xlim()
x_axis = np.linspace(xmin, xmax, 100)

# Calculate the probability density function (PDF) for the x-values.
plt.plot(x_axis, norm.pdf(x_axis, mu, sigma), 'red', linewidth=2, label='Gaussian Overlay')
plt.legend(); # Display the labels

## 4. Simhashing scheme and near-duplicate collection and verification

---

**Exercise 2:**  

Write the `simhash` function provided below. Make sure you understand that each bit of the simhash encodes which "side" the tf-idf document representation falls on a (pseudo-)random projection. 


---

In [ ]:
from tqdm import tqdm

## constants
K = 18
M = 3
B = M*K

def _simhash(id, b):
    """
        id is the document id, b is the desired length of the simhash
        it should return a bitstring of length b as explained in the
        first section of the notebook
    """
    simhash = np.zeros(b)
    for term in list(corpus[id].keys()):
        # print(term)
        ht = list(_termhash(term, b))
        # print(ht)
        ht_prime = np.array([1 if bit == "1" else -1 for bit in ht], dtype=float)
        # print(ht_prime)

        # print(ht_prime, corpus[id][term])
        simhash += ht_prime * corpus[id][term]
        # print(f"simhash: {simhash}")

    simhash_bit = [1 if entry >= 0 else 0 for entry in simhash]
    # print(f"simhash bit: {simhash_bit}")

    return "".join([str(sb) for sb in simhash_bit])

# simhash stores all simhashes of the whole corpus in a dictionary
simhash = {id: _simhash(id, B) for id in tqdm(corpus)}

---

**Exercise 3:**  

Based on simhashes, write code that places the documents in their corresponding lsh hash tables' buckets across $m$ repetitions. Once, the lsh hash tables have been
populated, find all potential near-duplicate candidate pairs. Check among all candidates their real cosine similarity and based on this determine true positives
(pairs of documents that are indeed similar and have collided in some table), and false positives (pairs of documents that are not similar but have collided in some table).
Compute speed of the method vs. false positives / true positives for different values of $m$ and $k$. Make sure that $k$ is not tiny (say greater than 10) if you want
the method to be fast.

---


In [ ]:
from pprint import pprint

def build_lsh_tables(simhash, m, k):
    tables = [dict() for _ in range(m)]
    for doc_id, hash_bits in simhash.items():
        for i in range(m):
            chunk = hash_bits[i*k : (i+1)*k]
            bucket_id = int(''.join(map(str, chunk)), 2)  # 0 .. 2^k - 1
            tables[i].setdefault(bucket_id, []).append(doc_id)
    return tables

tables = build_lsh_tables(simhash, M, K)
# pprint(tables[2])

In [ ]:
from itertools import combinations
from pprint import pprint

def collect_candidate_pairs(tables):
    candidates = set() # set notion to ensure no duplicates
    for table in tables:
        for bucket_id, doc_ids in table.items():
            if len(doc_ids) < 2:
                continue
            for a, b in combinations(sorted(doc_ids), 2):
                candidates.add((a, b))
    return candidates

candidate_pairs = collect_candidate_pairs(tables)
print(f"Candidate pairs count: {len(candidate_pairs)}")
pprint(list(candidate_pairs)[:5])

In [ ]:
import math
from pprint import pprint

def cosine_similarity(doc_vec_a, doc_vec_b):
    if not doc_vec_a or not doc_vec_b:
        return 0.0

    common_terms = set(doc_vec_a.keys()) & set(doc_vec_b.keys())
    dot = sum(doc_vec_a[t] * doc_vec_b[t] for t in common_terms)

    norm_a = math.sqrt(sum(w*w for w in doc_vec_a.values()))
    norm_b = math.sqrt(sum(w*w for w in doc_vec_b.values()))
    if norm_a == 0.0 or norm_b == 0.0:
        return 0.0
    return dot / (norm_a * norm_b)

In [ ]:
def classify_pairs(candidate_pairs, corpus, min_sim):
    true_positives = []
    false_positives = []
    for a, b in candidate_pairs:
        sim = cosine_similarity(corpus[a], corpus[b])
        if sim >= min_sim:
            true_positives.append((a, b, sim))
        else:
            false_positives.append((a, b, sim))
    # Sort (readability)
    true_positives.sort(key=lambda x: x[2], reverse=True)
    false_positives.sort(key=lambda x: x[2], reverse=True)
    return true_positives, false_positives

min_sim = 0.7
tp, fp = classify_pairs(candidate_pairs, corpus, min_sim=min_sim)
print(f"True positives (>= {min_sim}): {len(tp)} | False positives (< {min_sim}): {len(fp)}")
print("Top 5 TP:")
pprint(tp[:5])

In [ ]:
import time

# Benchmark speed vs TP/FP for different (M, K) configurations
lsh_configs = [
    (3, 12),
    (3, 18),
    (4, 16),
    (5, 12),
]

benchmark_results = []
for num_tables, bits_per_chunk in lsh_configs:
    total_bits = num_tables * bits_per_chunk
    t_start = time.perf_counter()

    # Recompute simhash for these parameters using the previously defined _simhash
    simhash_for_config = {doc_id: _simhash(doc_id, total_bits) for doc_id in corpus}

    lsh_tables = build_lsh_tables(simhash_for_config, num_tables, bits_per_chunk)
    candidate_pairs_for_config = collect_candidate_pairs(lsh_tables)
    true_positives_for_config, false_positives_for_config = classify_pairs(candidate_pairs_for_config, corpus, min_sim=0.8)
    elapsed_seconds = time.perf_counter() - t_start

    br = {
        "M": num_tables,
        "K": bits_per_chunk,
        "time_sec": round(elapsed_seconds, 3),
        "candidates": len(candidate_pairs_for_config),
        "tp": len(true_positives_for_config),
        "fp": len(false_positives_for_config),
        "precision": round(len(true_positives_for_config) / max(1, (len(true_positives_for_config) + len(false_positives_for_config))), 3),
    }

    pprint(br)

    benchmark_results.append(br)

In [ ]:
import matplotlib.pyplot as plt

def visualize_benchmarks(benchmark_results):
    if not benchmark_results:
        print("No benchmark results to visualize.")
        return

    labels = [f"M={br['M']}, K={br['K']}" for br in benchmark_results]
    times = [br['time_sec'] for br in benchmark_results]
    candidates = [br['candidates'] for br in benchmark_results]
    precision = [br['precision'] for br in benchmark_results]
    tp = [br['tp'] for br in benchmark_results]
    fp = [br['fp'] for br in benchmark_results]

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('LSH Simhash Benchmark Summary', fontsize=14)

    # Precision
    ax = axes[0][0]
    ax.bar(labels, precision, color="#4C9F70")
    ax.set_ylabel('Precision (TP / (TP+FP))')
    ax.set_ylim(0, 1)
    ax.set_title('Precision by (M, K)')
    ax.tick_params(axis='x', rotation=20)

    # Time
    ax = axes[0][1]
    ax.bar(labels, times, color="#3D5A80")
    ax.set_ylabel('Time (sec)')
    ax.set_title('Runtime by (M, K)')
    ax.tick_params(axis='x', rotation=20)

    # Candidates
    ax = axes[1][0]
    ax.bar(labels, candidates, color="#EE6C4D")
    ax.set_ylabel('# Candidate Pairs')
    ax.set_title('Candidates by (M, K)')
    ax.tick_params(axis='x', rotation=20)

    # TP vs FP stacked bar
    ax = axes[1][1]
    ax.bar(labels, tp, label='TP', color="#2A9D8F")
    ax.bar(labels, fp, bottom=tp, label='FP', color="#E76F51")
    ax.set_ylabel('Count (TP + FP)')
    ax.set_title('True vs False Positives')
    ax.legend()
    ax.tick_params(axis='x', rotation=20)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

visualize_benchmarks(benchmark_results)

## 5. Rules of delivery

- To be solved in _pairs_.

- No plagiarism; don't discuss your work with other teams. You can ask for help to others for simple things, such as recalling a python instruction or module, but nothing too specific to the session.

- If you feel you are spending much more time than the rest of the classmates, ask us for help. Questions can be asked either in person or by email, and you'll never be penalized by asking questions, no matter how stupid they look in retrospect.

- Write a short report listing the solutions to the exercises proposed. Include things like the important parts of your implementation (data structures used for representing objects, algorithms used, etc). You are welcome to add conclusions and findings that depart from what we asked you to do. We encourage you to discuss the difficulties you find; this lets us give you help and also improve the lab session for future editions.

- Turn the report to PDF. Make sure it has your names, date, and title. Include your code in your submission.

- Submit your work through the [raco](http://www.fib.upc.edu/en/serveis/raco.html); see date at the raco's submissions page.